# Importar las librerias

In [1]:
import os
import sys
root_dir = os.path.abspath(os.path.join(os.path.dirname('../pruebillas.ipynb'), '....'))
os.chdir(root_dir)

sys.path.insert(0, os.path.join(root_dir, 'src'))

In [2]:
from datetime import datetime
import MetaTrader5 as mt5
import pandas as pd
import pytz
import talib as ta
from backtesting import Strategy, Backtest

from utils import walk_forward
from utils import plot_full_equity_curve
from utils import get_wfo_stats
import numpy as np
from sklearn.linear_model import LinearRegression

pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1500)

# Obtener los datos

In [3]:

# display data on the MetaTrader 5 package
print("MetaTrader5 package author: ",mt5.__author__)
print("MetaTrader5 package version: ",mt5.__version__)
 
# establish connection to MetaTrader 5 terminal
if not mt5.initialize():
    print("initialize() failed, error code =",mt5.last_error())
    quit()
 
timezone = pytz.timezone("Etc/UTC")
utc_from = datetime(2014, 3, 1, tzinfo=timezone)
utc_to = datetime(2024, 9, 1, tzinfo=timezone)
rates = mt5.copy_rates_range("US500m", mt5.TIMEFRAME_H4, utc_from, utc_to)
 
mt5.shutdown()
 
df = pd.DataFrame(rates)

df['time'] = pd.to_datetime(df['time'], unit='s')
df = df.rename(columns={
    'time':'Date',
    'open':'Open',
    'high':'High',
    'low':'Low',
    'close':'Close',
})

df = df.set_index('Date')
df

MetaTrader5 package author:  MetaQuotes Ltd.
MetaTrader5 package version:  5.0.4288


,Open,High,Low,Close,tick_volume,spread,real_volume
Date,,,,,,,
2021-10-27 16:00:00,4581.26,4583.41,4580.60,4583.03,214,150,0
2021-10-28 08:00:00,4560.35,4570.04,4558.61,4565.60,1732,150,0
2021-10-28 12:00:00,4565.81,4593.79,4564.06,4584.57,5245,153,0
2021-10-28 16:00:00,4584.99,4596.47,4579.05,4595.30,3581,150,0
2021-10-28 20:00:00,4587.97,4588.18,4581.42,4583.43,921,157,0
...,...,...,...,...,...,...,...
2024-08-30 00:00:00,5603.08,5611.85,5596.51,5607.85,3894,149,0
2024-08-30 04:00:00,5607.85,5622.62,5605.09,5619.83,3586,150,0
2024-08-30 08:00:00,5619.58,5625.37,5614.72,5620.28,2526,159,0


# WFO

In [4]:
def diff_pips(price1, price2, pip_value, absolute=True):
    if absolute:
        difference = abs(price1 - price2)
    else:
        difference = price1 - price2
    pips = difference / pip_value
    
    return pips


class ShortMomentum(Strategy):
    risk = 1
    atr_multiplier = 1.5
    risk_reward = 1
    pip_value = 0.01
    bars_in_position = 0
    max_hold_period = 5
    
    def init(self):
        self.atr = self.I(ta.ATR, self.data.High, self.data.Low, self.data.Close)
        self.adx = self.I(ta.ADX, self.data.High, self.data.Low, self.data.Close )
        self.sma = self.I(ta.SMA, self.data.Close, timeperiod=100)
        
    def next(self):
        
        if self.position:
            self.bars_in_position += 1
            
            if self.bars_in_position >= self.max_hold_period:
                self.position.close()
                self.bars_in_position = 0
        
        else:
            if self.adx[-1] < 25:
                return
            
            go_long = self.data.Close[-1] > self.sma[-1]
            
            if go_long and self.data.Close[-1] > self.data.Close[-2]:
                
                sl = self.data.Close[-1] - self.atr_multiplier * self.atr[-1]
                tp = self.data.Close[-1] + self.atr_multiplier * self.atr[-1] * self.risk_reward
                
                sl_pips = diff_pips(self.data.Close[-1], sl, pip_value=self.pip_value, absolute=True)
                account_currency_risk = self.equity * (self.risk / 100)
                units = round(account_currency_risk / (self.pip_value * sl_pips))
                
                self.buy(
                    size=units, 
                    sl=sl,
                    tp=tp
                )
            
            if not go_long and self.data.Close[-1] < self.data.Close[-2]:
                
                sl = self.data.Close[-1] + self.atr_multiplier * self.atr[-1]
                tp = self.data.Close[-1] - self.atr_multiplier * self.atr[-1] * self.risk_reward
                
                sl_pips = diff_pips(self.data.Close[-1], sl, pip_value=self.pip_value, absolute=True)
                account_currency_risk = self.equity * (self.risk / 100)
                units = round(account_currency_risk / (self.pip_value * sl_pips))
                
                self.sell(
                    size=units, 
                    sl=sl,
                    tp=tp
                )

In [5]:


def  optim_func(series):
    return (series['Return [%]'] /  (1 + (-1*series['Max. Drawdown [%]']))) * np.log(1 + series['# Trades'])

def optim_func_2(stats):
    equity_curve = stats._equity_curve['Equity'].values    
    x = np.arange(len(equity_curve)).reshape(-1, 1)
    reg = LinearRegression().fit(x, equity_curve)
    stability_ratio = reg.score(x, equity_curve)
    
    return (stats['Return [%]'] /  (1 + (-1*stats['Max. Drawdown [%]']))) * np.log(1 + stats['# Trades']) * stability_ratio
    
    
lookback_bars = 1500
validation_bars = 300
warmup_bars = 200

params = {
    'atr_multiplier' : np.arange(2, 6, 0.5).tolist(),
    'risk_reward' : np.arange(0.4, 5, 0.2).tolist(),
    'maximize': optim_func_2
}

stats = walk_forward(
    ShortMomentum,
    df, 
    lookback_bars=lookback_bars,
    validation_bars=validation_bars,
    warmup_bars=warmup_bars, 
    params=params,
    commission=7e-4, 
    margin=1/30, 
    cash=100_000,
    verbose=True
)

plot_full_equity_curve(
    df, 
    stats, 
    warmup_bars=warmup_bars,
    lookback_bars=lookback_bars, 
    overlay_price=True,
)

wfo_stats = get_wfo_stats(stats, warmup_bars, df)

for k, v in wfo_stats.items():
    print(k, v)

train from 2021-10-27 16:00:00 to 2022-10-12 08:00:00


  0%|          | 0/8 [00:00<?, ?it/s]

validate from 2022-08-26 04:00:00 to 2022-12-20 12:00:00
{'atr_multiplier': 2.0, 'risk_reward': 1.4000000000000004}
equity final: 99465.95182558194
train from 2022-01-06 04:00:00 to 2022-12-20 12:00:00


  0%|          | 0/8 [00:00<?, ?it/s]

validate from 2022-11-04 04:00:00 to 2023-02-28 20:00:00
{'atr_multiplier': 2.0, 'risk_reward': 2.0000000000000004}
equity final: 102045.17094046816
train from 2022-03-15 12:00:00 to 2023-02-28 20:00:00


...

  0%|          | 0/8 [00:00<?, ?it/s]

validate from 2023-12-28 12:00:00 to 2024-04-23 16:00:00
{'atr_multiplier': 5.0, 'risk_reward': 0.6000000000000001}
equity final: 89884.3794571765
train from 2023-05-09 20:00:00 to 2024-04-23 16:00:00


  0%|          | 0/8 [00:00<?, ?it/s]

validate from 2024-03-06 20:00:00 to 2024-07-02 16:00:00
{'atr_multiplier': 5.0, 'risk_reward': 0.6000000000000001}
equity final: 88343.69405039732


Start 2021-10-27 16:00:00
End 2024-08-30 16:00:00
Duration 1038 days 00:00:00
Exposure Time [%] 8.950201884253028
Equity Final [$] 88343.69405039732
Equity Peak [$] 105832.61153576235
Return [%] -15.767175108933968
Buy & Hold Return [%] 23.219354880941225
Return (Ann.) [%] -7.764175571931853
Volatility (Ann.) [%] 7.83380331982437
Sharpe Ratio 0.0
Sortino Ratio 0.0
Calmar Ratio 0.0
Max. Drawdown [%] -21.127968808572017
Avg. Drawdown [%] -5.614035206753931
Max. Drawdown Duration 668 days 08:00:00
Avg. Drawdown Duration 168 days 08:00:00
# Trades 396
Win Rate [%] 44.94949494949495
Best Trade [%] 2.7087699816521593
Worst Trade [%] -1.9167868277715883
Avg. Trade [%] -0.026866294077043662
Max. Trade Duration 3 days 16:00:00
Avg. Trade Duration 1 days 00:00:00
Profit Factor 0.9135035807199894
Expectancy [%] -0.024268903325543562
SQN -0.993669584812715
